<a href="https://colab.research.google.com/github/ctrivino1/YLearn/blob/main/Frisch_Waugh_Theorem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# data
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import pandas as pd
#from ylearn.estimator_model.double_ml import DoubleML
from xgboost import XGBRegressor
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_predict

housing = fetch_california_housing(as_frame=True)
data = housing.frame
outcome = housing.target_names[0]
data[outcome] = housing.target
data.head()

In [ ]:
import seaborn as sns
sns.regplot(x="MedInc", y="MedHouseVal", data=data, ci=False, line_kws={'color':'r', 'label':'linear fit'})
plt.legend()
plt.title(f"Sales and coupon usage");

Normal Regression model

In [ ]:
import statsmodels.formula.api as smf
smf.ols('MedHouseVal ~ MedInc	+ HouseAge +	AveRooms +	AveBedrms +	Population +	AveOccup +	Latitude +	Longitude	', data).fit().summary().tables[1]

Now lets prove the Frisch-Waugh-Lowell theorem actually holds

In [ ]:
# Create a model that predicts medhouseval based on medincs residuals

data['MedInc_Residuals'] = smf.ols('MedInc ~ HouseAge +	AveRooms +	AveBedrms +	Population +	AveOccup +	Latitude +	Longitude', data).fit().resid

smf.ols('MedHouseVal ~ MedInc_Residuals - 1', data).fit().summary().tables[1]

In [ ]:
# Create a model that predicts medhouseval based on medincs residuals

data['HouseAge_tilde'] = smf.ols('HouseAge ~  MedInc +	AveRooms +	AveBedrms +	Population +	AveOccup +	Latitude +	Longitude', data).fit().resid

smf.ols('MedHouseVal ~ HouseAge_tilde -1', data).fit().summary().tables[1]

In [ ]:
# create primary model residuals
data['MedHouseVal_Residuals'] = smf.ols('MedHouseVal ~ AveRooms +	AveBedrms +	Population +	AveOccup +	Latitude +	Longitude', data).fit().resid
# create model that uses the residuals from the auxillary residuals to predict the primary models resdiduals
smf.ols('MedHouseVal_Residuals ~ MedInc_Residuals  + 1', data).fit().summary().tables[1]

In [ ]:

# Define the outcome variable (y) and treatment variable (T)
y = 'MedHouseVal'
x = 'MedInc'
v = data.columns.difference([y, x])

# Assume you have defined your ML models M_y and M_T
M_y = XGBRegressor()
M_x = XGBRegressor()

# Predict residuals using cross-validated predictions
data['y_residual'] = data[y] - cross_val_predict(M_y, data[v], data[y], cv=3)
data['x_residual'] = data[x] - cross_val_predict(M_x, data[v], data[x], cv=3)

# Fit the final ATE model using residuals
ATE_model = smf.ols(formula='y_residual ~ 1 + x_residual', data=data).fit()

# Display regression results
print(ATE_model.summary())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualize the treatment effect
sns.scatterplot(x=data[x], y=ATE_model.fittedvalues, color='blue', alpha=0.5)
plt.xlabel('MedInc (Treatment Variable)')
plt.ylabel('Predicted ATE')
plt.title('Scatter Plot of Predicted ATE against Treatment Variable')
plt.show()


In [ ]:

data["MedInc_hat"] =  smf.ols('MedInc ~ HouseAge +	AveRooms +	AveBedrms +	Population +	AveOccup +	Latitude +	Longitude', data).fit().predict()

ax = sns.regplot(x="MedHouseVal", y="MedInc", data=data, ci=False, line_kws={'color':'r', 'label':'linear fit'})
ax.vlines(data["MedHouseVal"],
   np.minimum(data["MedInc"], data["MedInc_hat"]),
   np.maximum(data["MedInc"], data["MedInc_hat"]),
   linestyle='--', color='k', alpha=0.5, linewidth=1, label="residuals")
plt.legend()
plt.title(f"MedInc/MedHouseVal residual model");

In [ ]:
sns.regplot(x="MedInc_Residuals", y="MedHouseVal_Residuals", data=data, ci=False, line_kws={'color':'r', 'label':'linear fit'})
plt.legend()
plt.title(f"Residual MedHouseVale and residual MedInc");

In [ ]:
data['MedHouseVal_Residuals_scaled'] = data['MedHouseVal_Residuals'] + np.mean(data['MedHouseVal'])
data['MedInc_Residuals_scaled'] = data['MedInc_Residuals'] + np.mean(data['MedInc'])

In [ ]:
sns.regplot(x="MedInc_Residuals_scaled", y="MedHouseVal_Residuals_scaled", data=data, ci=False, line_kws={'color':'r', 'label':'linear fit'})
plt.legend()
plt.title(f"Residual sales scaled and residual coupons scaled");